Iniciando bibliotecas e salvando informacoes

In [1]:
import zipfile
import os
import pandas as pd
import numpy as np

arquivo_zip = "data/HIST_PAINEL_COVIDBR_18nov2020.zip"
arquivo_csv = "data/HIST_PAINEL_COVIDBR_18nov2020.csv"

In [2]:

# Extração do arquivo zip
def extrair_dados_zip(zip):
  print("Extraindo arquivo ZIP...")
  data = zipfile.ZipFile(zip, "r")
  data = data.extractall("data")
  
# Ler dados da planilha
def ler_dados_csv(csv):
  print("Lendo a planilha...")
  data = pd.read_csv(csv, encoding="utf-8",delimiter=";")
  return data

# Chamando função caso não tenha extraido
if not os.path.exists(f"{arquivo_csv}"):
  extrair_dados_zip(f"{arquivo_zip}")

# Lendo planilha
data = ler_dados_csv(arquivo_csv)

Lendo a planilha...


In [49]:
print(data.iloc[55])

regiao                                 Brasil
estado                                    NaN
municipio                                 NaN
coduf                                      76
codmun                                    NaN
codRegiaoSaude                            NaN
nomeRegiaoSaude                           NaN
data                      2020-04-20 00:00:00
semanaEpi                                  17
populacaoTCU2019                  210147125.0
casosAcumulado                          40581
casosNovos                               1927
obitosAcumulado                          2575
obitosNovos                               113
Recuperadosnovos                      22991.0
emAcompanhamentoNovos                 15015.0
interior/metropolitana                    NaN
Name: 55, dtype: object


In [137]:
def relatorio_meses(data, eh_estado, relatorio_brasil):
  lista_meses = []
  casos_acumulados = 0
  obitos_acumulados = 0

  for mes in range(2,12):
    data_mes = data[data["data"].dt.month == mes]
    casos = data_mes["casosNovos"].sum()
    obitos = data_mes["obitosNovos"].sum()
    infectados = data_mes.iloc[-1]["emAcompanhamentoNovos"]
    
    casos_acumulados += casos
    obitos_acumulados += obitos
    
    mes = int(mes)
    variacao_casos = 0
    variacao_obitos = 0

    mortalidade = round((obitos / casos) * 100 if casos != 0 else 0,2)
    
    if mes > 2:
      casos_antigo = lista_meses[mes-3][1]
      obitos_antigo = lista_meses[mes-3][2]
      
      variacao_casos =  (casos / casos_antigo) - 1 if casos_antigo != 0 else casos
      variacao_casos *= 100
      variacao_casos = round(variacao_casos, 2)
      
      variacao_obitos = (obitos / obitos_antigo) - 1 if obitos_antigo != 0 else obitos
      variacao_obitos *= 100
      variacao_obitos = round(variacao_obitos, 2)
      
    
    colunas = ["mes", "casosNovos", "obitosNovos", "mortalidade", "infectados", "variacaoDeCasos" ,"variacaoDeObitos", "casosAcumulados", "obitosAcumulados"]
    dados_colunas = [mes, casos, obitos, mortalidade, infectados, variacao_casos, variacao_obitos, casos_acumulados, obitos_acumulados]
    
    if eh_estado:
      colunas += "Comparação Nacional (Casos)", "Comparação Nacional (Obitos)"
      relatorio_mes = relatorio_brasil[relatorio_brasil.index == mes]
      comparacao_nacional_casos = casos / relatorio_mes["casosNovos"].sum()
      comparacao_nacional_casos *=  100
      comparacao_nacional_casos = round(comparacao_nacional_casos, 2) 
      
      comparacao_nacional_obitos = obitos / relatorio_mes["obitosNovos"].sum()
      comparacao_nacional_obitos *=  100
      comparacao_nacional_obitos = round(comparacao_nacional_obitos, 2) 
      
      dados_colunas+= comparacao_nacional_casos, comparacao_nacional_obitos
      
    
    lista_meses.append(dados_colunas)

  data_meses = pd.DataFrame(lista_meses, columns=colunas)
  data_meses = data_meses.set_index("mes")
  return data_meses

## Retirando dados sobre o brasil

In [3]:
data["data"] = pd.to_datetime(data["data"], format="%Y-%m-%d")

print(data.dtypes)

regiao                            object
estado                            object
municipio                         object
coduf                              int64
codmun                           float64
codRegiaoSaude                   float64
nomeRegiaoSaude                   object
data                      datetime64[ns]
semanaEpi                          int64
populacaoTCU2019                 float64
casosAcumulado                     int64
casosNovos                         int64
obitosAcumulado                    int64
obitosNovos                        int64
Recuperadosnovos                 float64
emAcompanhamentoNovos            float64
interior/metropolitana           float64
dtype: object


In [4]:
# Dados do brasil
data_brasil = data[data["regiao"] == "Brasil"]

In [5]:
print(data_brasil.iloc[0])

regiao                                 Brasil
estado                                    NaN
municipio                                 NaN
coduf                                      76
codmun                                    NaN
codRegiaoSaude                            NaN
nomeRegiaoSaude                           NaN
data                      2020-02-25 00:00:00
semanaEpi                                   9
populacaoTCU2019                  210147125.0
casosAcumulado                              0
casosNovos                                  0
obitosAcumulado                             0
obitosNovos                                 0
Recuperadosnovos                          NaN
emAcompanhamentoNovos                     NaN
interior/metropolitana                    NaN
Name: 0, dtype: object


In [6]:
# A maior quantidade de novos casos no Brasil
maximo_casos_novos_brasil = data_brasil["casosNovos"].max()
data_casos_brasil = data_brasil[data_brasil["casosNovos"] == maximo_casos_novos_brasil]
maximo_casos_brasil = data_casos_brasil["casosNovos"]
date_casos_brasil = data_casos_brasil["data"]
print(data_casos_brasil.iloc[0])

regiao                                 Brasil
estado                                    NaN
municipio                                 NaN
coduf                                      76
codmun                                    NaN
codRegiaoSaude                            NaN
nomeRegiaoSaude                           NaN
data                      2020-07-29 00:00:00
semanaEpi                                  31
populacaoTCU2019                  210147125.0
casosAcumulado                        2552265
casosNovos                              69074
obitosAcumulado                         90134
obitosNovos                              1595
Recuperadosnovos                    1786753.0
emAcompanhamentoNovos                675378.0
interior/metropolitana                    NaN
Name: 155, dtype: object


In [7]:
# A maior quantidade de novos obitos no Brasil
maximo_obitos_novos_brasil = data_brasil["obitosNovos"].max()
data_obitos_brasil = data_brasil[data_brasil["obitosNovos"] == maximo_obitos_novos_brasil]
maximo_obitos_brasil = data_obitos_brasil["obitosNovos"]
date_obitos_brasil = data_obitos_brasil["data"]
print(data_obitos_brasil.iloc[0])

regiao                                 Brasil
estado                                    NaN
municipio                                 NaN
coduf                                      76
codmun                                    NaN
codRegiaoSaude                            NaN
nomeRegiaoSaude                           NaN
data                      2020-07-29 00:00:00
semanaEpi                                  31
populacaoTCU2019                  210147125.0
casosAcumulado                        2552265
casosNovos                              69074
obitosAcumulado                         90134
obitosNovos                              1595
Recuperadosnovos                    1786753.0
emAcompanhamentoNovos                675378.0
interior/metropolitana                    NaN
Name: 155, dtype: object


In [113]:
# Ver casos e obitos de todos os meses
relatorio_brasil = relatorio_meses(data_brasil, False, False)

In [114]:
print("Meses ordenados com o mês ordenado")
print(relatorio_brasil)

Meses ordenados com o mês ordenado
     casosNovos  obitosNovos  mortalidade  infectados  variacaoDeCasos  \
mes                                                                      
2             2            0         0.00         NaN             0.00   
3          5715          201         3.52         NaN        285650.00   
4         79663         5700         7.16     43544.0          1293.93   
5        428820        23413         5.46    278331.0           438.29   
6        887841        30280         3.41    552407.0           107.04   
7       1260444        32881         2.61    725959.0            41.97   
8       1245787        28906         2.32    689157.0            -1.16   
9        902663        22571         2.50    486607.0           -27.54   
10       724670        15932         2.20    402823.0           -19.72   
11       484559         8729         1.80    429449.0           -33.13   

     variacaoDeObitos  casosAcumulados  obitosAcumulados  
mes              

In [102]:
print("Ordenado com base nos obitos")
print(relatorio_brasil.sort_values("obitosNovos", ascending=False).iloc[:, [0,1,2,3,4,5]])

Ordenado com base nos obitos
     casosNovos  obitosNovos  mortalidade  infectados  variacaoDeCasos  \
mes                                                                      
7       1260444        32881         2.61    725959.0            41.97   
6        887841        30280         3.41    552407.0           107.04   
8       1245787        28906         2.32    689157.0            -1.16   
5        428820        23413         5.46    278331.0           438.29   
9        902663        22571         2.50    486607.0           -27.54   
10       724670        15932         2.20    402823.0           -19.72   
11       484559         8729         1.80    429449.0           -33.13   
4         79663         5700         7.16     43544.0          1293.93   
3          5715          201         3.52         NaN        285650.00   
2             2            0         0.00         NaN             0.00   

     variacaoDeObitos  
mes                    
7                8.59  
6         

In [97]:
print("Ordenado com base nos casos")
print(relatorio_brasil.sort_values("casosNovos", ascending=False).iloc[:, [0,1,2,3,4,5]])


Ordenado com base nos casos
     casosNovos  obitosNovos  mortalidade  infectados  variacaoDeCasos
mes                                                                   
7       1260444        32881         2.61    725959.0            41.97
8       1245787        28906         2.32    689157.0            -1.16
9        902663        22571         2.50    486607.0           -27.54
6        887841        30280         3.41    552407.0           107.04
10       724670        15932         2.20    402823.0           -19.72
11       484559         8729         1.80    429449.0           -33.13
5        428820        23413         5.46    278331.0           438.29
4         79663         5700         7.16     43544.0          1293.93
3          5715          201         3.52         NaN        285650.00
2             2            0         0.00         NaN             0.00


In [103]:
print("Ordenado com base na mortalidade")
print(relatorio_brasil.sort_values("mortalidade", ascending=False).iloc[:, [0,1,2,3,4,5]])

Ordenado com base na mortalidade
     casosNovos  obitosNovos  mortalidade  infectados  variacaoDeCasos  \
mes                                                                      
4         79663         5700         7.16     43544.0          1293.93   
5        428820        23413         5.46    278331.0           438.29   
3          5715          201         3.52         NaN        285650.00   
6        887841        30280         3.41    552407.0           107.04   
7       1260444        32881         2.61    725959.0            41.97   
9        902663        22571         2.50    486607.0           -27.54   
8       1245787        28906         2.32    689157.0            -1.16   
10       724670        15932         2.20    402823.0           -19.72   
11       484559         8729         1.80    429449.0           -33.13   
2             2            0         0.00         NaN             0.00   

     variacaoDeObitos  
mes                    
4             2735.82  
5     

In [99]:
relatorio_brasil.to_csv("data/data_for_months.csv" )

# Lendo dados pela região

In [14]:
aleatoria = data[data["regiao"] != "Brasil"].iloc[1900]
print(aleatoria)


regiao                               Nordeste
estado                                     MA
municipio                                 NaN
coduf                                      21
codmun                                    NaN
codRegiaoSaude                            NaN
nomeRegiaoSaude                           NaN
data                      2020-03-06 00:00:00
semanaEpi                                  10
populacaoTCU2019                    7075181.0
casosAcumulado                              0
casosNovos                                  0
obitosAcumulado                             0
obitosNovos                                 0
Recuperadosnovos                          NaN
emAcompanhamentoNovos                     NaN
interior/metropolitana                    NaN
Name: 2170, dtype: object


In [125]:
print(aleatoria["municipio"])

nan


In [139]:
regioes = data["regiao"].unique()[1:]

relatorios_regioes = {}

for regiao in regioes:
  data_regiao = data[data["regiao"] == regiao]
  data_regiao = data_regiao[data_regiao["municipio"].isna()]
  relatorio = relatorio_meses(data_regiao, True, relatorio_brasil)
  relatorios_regioes[regiao] = relatorio
  
  


C:\Users\Vitor\AppData\Local\Temp\ipykernel_9024\3874453886.py:44: RuntimeWarning: invalid value encountered in scalar divide
  comparacao_nacional_obitos = obitos / relatorio_mes["obitosNovos"].sum()


In [140]:
# Norte
print(relatorios_regioes["Norte"])

     casosNovos  obitosNovos  mortalidade  infectados  variacaoDeCasos  \
mes                                                                      
2             0            0         0.00         NaN             0.00   
3           294            4         1.36         NaN         29400.00   
4         10482          708         6.75         NaN          3465.31   
5         97019         4979         5.13         NaN           825.58   
6        154875         3836         2.48         NaN            59.63   
7        145433         2323         1.60         NaN            -6.10   
8        129687         1626         1.25         NaN           -10.83   
9         93804         1570         1.67         NaN           -27.67   
10        71964          932         1.30         NaN           -23.28   
11        43537          523         1.20         NaN           -39.50   

     variacaoDeObitos  casosAcumulados  obitosAcumulados  \
mes                                                

In [141]:
# Nordest
print(relatorios_regioes["Nordeste"])

     casosNovos  obitosNovos  mortalidade  infectados  variacaoDeCasos  \
mes                                                                      
2             0            0         0.00         NaN             0.00   
3           943           23         2.44         NaN         94300.00   
4         23675         1518         6.41         NaN          2410.60   
5        155680         7344         4.72         NaN           557.57   
6        301891        10436         3.46         NaN            93.92   
7        381418         9380         2.46         NaN            26.34   
8        287974         6527         2.27         NaN           -24.50   
9        178795         4026         2.25         NaN           -37.91   
10       157041         2985         1.90         NaN           -12.17   
11        88146         1527         1.73         NaN           -43.87   

     variacaoDeObitos  casosAcumulados  obitosAcumulados  \
mes                                                

In [142]:
# Centro-oeste
print(relatorios_regioes["Centro-Oeste"])

     casosNovos  obitosNovos  mortalidade  infectados  variacaoDeCasos  \
mes                                                                      
2             0            0         0.00         NaN             0.00   
3           470            5         1.06         NaN         47000.00   
4          2229           76         3.41         NaN           374.26   
5         14743          297         2.01         NaN           561.42   
6         79869         1363         1.71         NaN           441.74   
7        153723         3579         2.33         NaN            92.47   
8        183797         3908         2.13         NaN            19.56   
9        159947         3408         2.13         NaN           -12.98   
10        99160         2190         2.21         NaN           -38.00   
11        45957          970         2.11         NaN           -53.65   

     variacaoDeObitos  casosAcumulados  obitosAcumulados  \
mes                                                

In [143]:
# Sul
print(relatorios_regioes["Sul"])

     casosNovos  obitosNovos  mortalidade  infectados  variacaoDeCasos  \
mes                                                                      
2             0            0         0.00         NaN             0.00   
3           680            9         1.32         NaN         68000.00   
4          4334          171         3.95         NaN           537.35   
5         18088          369         2.04         NaN           317.35   
6         53788         1068         1.99         NaN           197.37   
7        152600         3303         2.16         NaN           183.71   
8        212855         4104         1.93         NaN            39.49   
9        151428         3085         2.04         NaN           -28.86   
10       135411         2070         1.53         NaN           -10.58   
11       142899         1587         1.11         NaN             5.53   

     variacaoDeObitos  casosAcumulados  obitosAcumulados  \
mes                                                

In [144]:
# Sudeste
print(relatorios_regioes["Sudeste"])

     casosNovos  obitosNovos  mortalidade  infectados  variacaoDeCasos  \
mes                                                                      
2             2            0         0.00         NaN             0.00   
3          3462          162         4.68         NaN        173000.00   
4         39004         3232         8.29         NaN          1026.63   
5        145369        10448         7.19         NaN           272.70   
6        299013        13640         4.56         NaN           105.69   
7        432735        14349         3.32         NaN            44.72   
8        438057        12810         2.92         NaN             1.23   
9        322756        10457         3.24         NaN           -26.32   
10       264320         7796         2.95         NaN           -18.11   
11       167051         4133         2.47         NaN           -36.80   

     variacaoDeObitos  casosAcumulados  obitosAcumulados  \
mes                                                